In [2]:
import requests
import hashlib
import time
import json

In [42]:
API_KEY = "4YCF385ZNXLEHADRUSFV"
API_SECRET = "YcjAfd9x7S$$qX^4u#n$rvLD$X6c^pT87ShWvF3V"

In [48]:
# Generate the timestamp
timestamp = str(int(time.time()))
auth_hash = hashlib.sha1((API_KEY + API_SECRET + timestamp).encode('utf-8')).hexdigest()

headers = {
    "X-Auth-Date": timestamp,
    "X-Auth-Key": API_KEY,
    "Authorization": auth_hash,
    "User-Agent": "PodcastIndex-Client"
}

# Step 1: Get Trending Podcasts
url = "https://api.podcastindex.org/api/1.0/podcasts/trending"
params = {"max": 200}  # Get up to 200 trending podcasts

response = requests.get(url, headers=headers, params=params)
trending_podcasts = response.json().get("feeds", [])

print(f"Retrieved {len(trending_podcasts)} trending podcasts.")

Retrieved 200 trending podcasts.


In [49]:

# Step 1: Download trending podcasts
url = "https://api.podcastindex.org/api/1.0/podcasts/trending"
response = requests.get(url, headers=headers, params=params)
trending_data = response.json()

# Step 2: Extract relevant fields
trending_data = response.json()
trending_podcasts = trending_data.get("feeds", [])

simplified_podcasts = [
    {
        "title": feed.get("title"),
        "url": feed.get("url"),
        "description": feed.get("description"),
        "id": feed.get("id"),
        "author": feed.get("author"),
    }
    for feed in trending_podcasts
]


# Step 3: Save to JSON file
output_path = "top_podcasts.json"
with open(output_path, "w") as f:
    json.dump(simplified_podcasts, f, indent=4)

print(f"Saved {len(simplified_podcasts)} podcasts to {output_path}")




Saved 200 podcasts to top_podcasts.json


In [50]:
with open("top_podcasts.json", "r") as f:
    podcasts = json.load(f)

all_episodes = []

for podcast in podcasts:
    feed_id = podcast.get("id")
    podcast_title = podcast.get("title")

    print(f"Fetching episodes for: {podcast_title}")

    response = requests.get(
        "https://api.podcastindex.org/api/1.0/episodes/byfeedid",
        headers=headers,
        params= {"id": feed_id, "max": 1} #more episodes from one podcast - change max
    )

    if response.status_code == 200:
        episodes = response.json().get("items", [])
        for ep in episodes:
            all_episodes.append({
                "podcast": podcast_title,
                "episode_title": ep.get("title"),
                "audio_url": ep.get("enclosureUrl"),
                "id": ep.get("id")
            })
    

# Save to top_episodes.json
with open("top_episodes.json", "w") as f:
    json.dump(all_episodes, f, indent=4)

print(f"✅ Saved {len(all_episodes)} episodes to top_episodes.json")


Fetching episodes for: Calming Anxiety
Fetching episodes for: Spirit Force
Fetching episodes for: Ethan Brings You the World
Fetching episodes for: Promo Estereo Podcast
Fetching episodes for: Craave Raasi Palan
Fetching episodes for: A hombros de gigantes
Fetching episodes for: 高詩佳故事學堂│經典 ‧ 智慧 ‧ 閱讀素養
Fetching episodes for: このは屋
Fetching episodes for: このは屋
Fetching episodes for: High Performance Mindset | Learn from World-Class Leaders, Consultants, Athletes & Coaches about Mindset
Fetching episodes for: PHNX Arizona Cardinals Podcast
Fetching episodes for: Eight Years in Paris
Fetching episodes for: The Politics Prepping and Paranormal Podcast
Fetching episodes for: It's a Hard Knicks Life
Fetching episodes for: The John Batchelor Show
Fetching episodes for: Scott Ritter Extra: Ask the Inspector
Fetching episodes for: WRFH/Radio Free Hillsdale 101.7 FM
Fetching episodes for: The Young Turks
Fetching episodes for: The Vibe You're In
Fetching episodes for: Real America’s Voice
Fetching 

In [ ]:
import subprocess
import os
output_dir = "transcripts"
os.makedirs(output_dir, exist_ok=True)

# Load episodes file
with open("top_episodes.json", "r") as f:
    episodes = json.load(f)

# Iterate through each podcast entry
for episode in episodes:
    podcast_name = episode.get("podcast", "unknown_podcast")
    episode_title = episode.get("episode_title", "unknown_episode")
    audio_url = episode.get("audio_url")

    # Construct output filename 
    safe_title = episode_title.replace(" ", "_").replace("/", "_")
    safe_podcast = podcast_name.replace(" ", "_").replace("/", "_")
    output_file = os.path.join(output_dir, f"{safe_podcast}_{safe_title}.json")

    # Check if the audio_url is valid
    if audio_url and audio_url.strip():
        print(f"Transcribing {episode_title} from {audio_url}")
        try:
            # Run podscript command
            result = subprocess.run(
                [
                    "/Users/terezasaskova/go/bin/podscript",
                     "assembly-ai",
                     "--from-url", audio_url,
                    "--output", output_file
                ],
                check=True,
                capture_output=True,
                text=True
            )

            print(f"Transcript saved to {output_file}")

        except subprocess.CalledProcessError as e:
            print(f"Subprocess error:\n{e.stderr}")
        except Exception as e:
            print(f"Exception occurred: {e}")
    else:
        print(f"No valid audio URL found for {episode_title}")

print("Transcription process complete.")


Transcribing I nurture balance in all I think, feel and do from https://dts.podtrac.com/redirect.mp3/api.spreaker.com/download/episode/66343448/i_nurture_balance_in_all_i_think_feel_and_do.mp3
Transcript saved to transcripts/Calming_Anxiety_I_nurture_balance_in_all_I_think,_feel_and_do.json
Transcribing Adventures of Captain Epoch Ep 2-6: The Fed and the Titanic from https://dts.podtrac.com/redirect.mp3/api.spreaker.com/download/episode/66345491/adventures_of_captain_epoch_ep_2_6_thefedandthetitanic.mp3
Transcript saved to transcripts/Spirit_Force_Adventures_of_Captain_Epoch_Ep_2-6:_The_Fed_and_the_Titanic.json
Transcribing 05-30-25 Part 12 from https://dts.podtrac.com/redirect.mp3/api.spreaker.com/download/episode/66344036/untitled_session_105_mixdown_12.mp3
Transcript saved to transcripts/Ethan_Brings_You_the_World_05-30-25_Part_12.json
Transcribing AÑORANZAS MUSICALES || PROGRAMA DE ESTRENO from https://dts.podtrac.com/redirect.mp3/api.spreaker.com/download/episode/66345135/a_oranza

In [16]:
import whisper
import os
import subprocess
from pathlib import Path
import sys
import tempfile


# Load Whisper model once
model = whisper.load_model("base")  # or "tiny", "small"

# Paths
AUDIO_DIR = Path("audio")
TRANSCRIPT_DIR = Path("transcripts")
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(TRANSCRIPT_DIR, exist_ok=True)

def download_audio(url, output_path):
    subprocess.run([
        "yt-dlp",
        "-x", "--audio-format", "mp3",
        "-o", str(output_path),
        url
    ], check=True)

def transcribe(audio_path):
    result = model.transcribe(str(audio_path))
    return result["text"]

In [ ]:
import json
import os
import subprocess
from pathlib import Path
import whisper

# Load Whisper model once
model = whisper.load_model("base")  # You can choose tiny, small, medium, large

# Directories
AUDIO_DIR = Path("audio")
TRANSCRIPT_DIR = Path("transcripts")
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(TRANSCRIPT_DIR, exist_ok=True)

# Load episodes
with open("top_episodes.json", "r") as f:
    episodes = json.load(f)

# Loop through each episode
for episode in episodes:
    podcast_title = episode.get("podcast", "unknown_podcast")
    episode_title = episode.get("episode_title", "unknown_episode")
    audio_url = episode.get("audio_url")

    # Safety-check the URL
    if not audio_url or not audio_url.strip():
        print(f"❌ No valid audio URL for: {episode_title}")
        continue

    print(f"🎧 Transcribing: {podcast_title} - {episode_title}")

    # Create safe filenames
    safe_title = episode_title.replace(" ", "_").replace("/", "_")
    safe_podcast = podcast_title.replace(" ", "_").replace("/", "_")
    filename_base = f"{safe_podcast}_{safe_title}"
    audio_path = AUDIO_DIR / f"{filename_base}.mp3"
    transcript_path = TRANSCRIPT_DIR / f"{filename_base}.json"

    # Skip if transcript already exists
    if transcript_path.exists():
        print(f"✅ Transcript already exists: {transcript_path}")
        continue

    # 1. Download audio with yt-dlp
    try:
        subprocess.run([
            "yt-dlp",
            "-x", "--audio-format", "mp3",
            "-o", str(audio_path),
            audio_url
        ], check=True)
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to download audio: {e}")
        continue

    # 2. Transcribe with Whisper
    try:
        result = model.transcribe(str(audio_path))
        with open(transcript_path, "w") as f:
            f.write(result["text"])
        print(f"📝 Transcript saved: {transcript_path}")
    except Exception as e:
        print(f"❌ Whisper transcription error: {e}")


🎧 Transcribing: Calming Anxiety - I nurture balance in all I think, feel and do
[generic] Extracting URL: https://dts.podtrac.com/redirect.mp3/api.spreaker.com/download/episode/66343448/i_nurture_balance...hink_feel_and_do.mp3
[generic] i_nurture_balance_in_all_i_think_feel_and_do: Downloading webpage
[redirect] Following redirect to https://d1bxy2pveef3fq.cloudfront.net/v1/variant/a186d864-09a7-5c0f-b38f-a782a61790a1.mp3?session_id=467bbc1b-30ba-58ad-af37-09f3e4b5a28f&ab=320&al=634017&ao=0&cc=NYNNNNYYYY&ct=DOWNLOAD&episode_id=66343448&show_id=4110266&user_id=11660338&organization_id=8806749&tenant=SPREAKER&timestamp=1748785557&ppi=060da7e3-596a-5c94-82c3-34336b3abaf1&epi=05021f14-4b5f-5559-9d82-dd39e12d5d17&media_type=dynamic&fallback_url=https%3A%2F%2Fd1bxy2pveef3fq.cloudfront.net%2Fepisodes%2Foriginal%2F50469162%3Fsession_id%3D467bbc1b-30ba-58ad-af37-09f3e4b5a28f%26ab%3D320%26al%3D634017%26ao%3D0%26cc%3DNYNNNNYYYY%26ct%3DDOWNLOAD%26episode_id%3D66343448%26show_id%3D4110266%26user_id

/opt/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
